In [1]:
experimentsPath = "/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = "CIFAR_phylogeny_from_scratch_2"
device = 1
detailed_reporting = False

In [ ]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
from tqdm.auto import trange
import wandb

import warnings
# warnings.filterwarnings("ignore")

try:
    import wandb
except:
    print('wandb not found')

from myhelpers import config_plots, TrialStatistics
from myhelpers.try_warning import try_running
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()

experimetnsFileName = "experiments.csv"
WANDB_message="wandb not working"


# For logging to server
try_running(lambda : wandb.login(), WANDB_message)

experimentPathAndName = os.path.join(experimentsPath, experimentName)
# set cuda
if device is not None:
    print("using cuda", device)
    torch.cuda.set_device(device)

else:
    print("using cpu")

# get experiment params
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

# init experiments file
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  
experiment_index = 0

# Loop through experiments
# with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        if experiment_params['image_path'] == 'cifar-100-python':
            datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
        else:
            datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        architecture = {
            "fine": len(train_loader.dataset.csv_processor.getFineList()),
            "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
        }

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)

            run = try_running(lambda : wandb.init(project='HGNN', group=experimentName+"-"+experimentHash, name=trialHash, config=row_information), WANDB_message) #, reinit=True

            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)
#             print(model)

#             try_running(lambda : wandb.watch(model, log="all"), WANDB_message)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                initModelPath = CNN.getInitModelFile(experimentPathAndName)
                if os.path.exists(initModelPath):
                    model.load_state_dict(torch.load(initModelPath))
                    print("Init Model {0} found!".format(initModelPath))
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=detailed_reporting)

            # Add to experiments file
            if os.path.exists(experimentsFileNameAndPath):
                experiments_df = pd.read_csv(experimentsFileNameAndPath)
            else:
                experiments_df = pd.DataFrame()

            record_exists = not (experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName]).empty if not experiments_df.empty else False
            if record_exists:
                experiments_df.drop(experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName].index, inplace = True) 

            experiments_df = experiments_df.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
            experiments_df.to_csv(experimentsFileNameAndPath, header=True, index=False)

            try_running(lambda : run.finish(), WANDB_message)

        bar.update()

        experiment_index = experiment_index + 1



# if __name__ == "__main__":
#     torch.multiprocessing.set_start_method('spawn')
    
#     import argparse

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--cuda', required=True, type=int)
#     parser.add_argument('--experiments', required=True)
#     parser.add_argument('--data', required=True)
#     parser.add_argument('--name', required=True)
#     parser.add_argument('--detailed', required=False, action='store_true')
#     args = parser.parse_args()
#     main(experimentName=args.name, experimentsPath=args.experiments, dataPath=args.data, device=args.cuda, detailed_reporting=args.detailed)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)
experiment:   0%|          | 0/4 [00:00<?, ?it/s]

using cuda 1
{'image_path': 'cifar-100-python', 'suffix': None, 'img_res': 32, 'augmented': True, 'batchSize': 128, 'learning_rate': 0.1, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': False, 'epochs': 200, 'patience': -1, 'modelType': 'BB', 'lambda': 10, 'tl_model': 'preResNet', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.1, 'tripletEnabled': False, 'tripletSamples': 3, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'phylogeny_loss': False, 'displayName': 'CIFAR preresnet from scratch no triplet', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}
Creating dataset...
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Creating dataset... Done.
Loading saved indices...
Creating loaders...
Creating loaders... Done.


{'experimentName': 'CIFAR_phylogeny_from_scratch_2', 'modelName': 'models/c872f5b8a4fd28ecbcdea6165e24d5e2db74486ccf13af62f314d81f', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': '3599eece106c56ef2f5bb246d59ac03857454f60ff22d9529d143466', 'trialHash': 'c872f5b8a4fd28ecbcdea6165e24d5e2db74486ccf13af62f314d81f', 'image_path': 'cifar-100-python', 'suffix': None, 'img_res': 32, 'augmented': True, 'batchSize': 128, 'learning_rate': 0.1, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': False, 'epochs': 200, 'patience': -1, 'modelType': 'BB', 'lambda': 10, 'tl_model': 'preResNet', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.1, 'tripletEnabled': False, 'tripletSamples': 3, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'phylogeny_loss': False, 'displayName': 'CIFAR preresnet from scratch no triplet', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}


wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


layer4 is not in ['conv1', 'layer1', 'layer2', 'layer3', 'bn', 'relu', 'avgpool']




iteration:   0%|          | 0/200 [00:00<?, ?it/s]

Training started...


/home/elhamod/melhamodenv3/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


iteration:   0%|          | 0/200 [00:47<?, ?it/s, min_val_loss=inf, train=0.00133, val=0.00122, val_loss=4.61]

iteration:   0%|          | 1/200 [00:47<2:36:17, 47.12s/it, min_val_loss=inf, train=0.00133, val=0.00122, val_loss=4.61]

iteration:   0%|          | 1/200 [02:50<2:36:17, 47.12s/it, min_val_loss=inf, train=0.0857, val=0.0795, val_loss=4.56]  

iteration:   1%|          | 2/200 [02:50<3:51:01, 70.01s/it, min_val_los

In [ ]:
print(model)